In [1]:
!pip install mtcnn tensorflow foolbox torch torchvision facenet-pytorch numpy matplotlib



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 845.6 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.

In [2]:
!wget https://drive.google.com/uc?id=1M8R33eK9ZTk4N0dHz6MjFB0mKe3wAfg5 -O celebA.zip
!unzip celebA.zip


--2025-03-18 18:38:43--  https://drive.google.com/uc?id=1M8R33eK9ZTk4N0dHz6MjFB0mKe3wAfg5
Resolving drive.google.com (drive.google.com)... 173.194.217.100, 173.194.217.138, 173.194.217.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1M8R33eK9ZTk4N0dHz6MjFB0mKe3wAfg5 [following]
--2025-03-18 18:38:43--  https://drive.usercontent.google.com/download?id=1M8R33eK9ZTk4N0dHz6MjFB0mKe3wAfg5
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.11.132, 2607:f8b0:400c:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.11.132|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-18 18:38:43 ERROR 404: Not Found.

Archive:  celebA.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of 

In [3]:
!pip install lz4


In [4]:
!pip uninstall -y lz4
!pip install --no-cache-dir lz4

Found existing installation: lz4 4.4.3
Uninstalling lz4-4.4.3:
  Successfully uninstalled lz4-4.4.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00


In [5]:
!pip install numpy scipy torch torchvision mtcnn facenet-pytorch foolbox lz4


In [6]:
!pip install mtcnn


In [7]:
from mtcnn import MTCNN
from PIL import Image
import numpy as np

detector = MTCNN()

def extract_face(image_path):
    image = Image.open(image_path).convert("RGB")
    pixels = np.array(image)
    results = detector.detect_faces(pixels)

    if results:
        x, y, width, height = results[0]['box']
        face = pixels[y:y+height, x:x+width]
        face_image = Image.fromarray(face).resize((160, 160))
        return np.array(face_image)
    return None


In [8]:
import torch

def preprocess_face(face):
    face = face.astype("float32") / 255.0  # Normalize pixel values
    face = torch.tensor(face).permute(2, 0, 1)  # Convert to PyTorch format (C, H, W)
    return face.unsqueeze(0)  # Add batch dimension


In [9]:
!pip install facenet-pytorch


In [10]:
from facenet_pytorch import InceptionResnetV1

facenet = InceptionResnetV1(pretrained="vggface2").eval()


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [11]:
def get_face_embedding(face_tensor):
    return facenet(face_tensor).detach().numpy()


In [12]:
!pip install foolbox


In [13]:
import foolbox as fb

# Create a Foolbox model wrapper
fmodel = fb.PyTorchModel(facenet, bounds=(0, 1))

# Define FGSM attack
attack = fb.attacks.FGSM()

def generate_fgsm_attack(face_tensor):
    raw_adversarial, clipped_adversarial, _ = attack(fmodel, face_tensor, criterion=fb.criteria.Misclassification())
    return clipped_adversarial


In [14]:
!pip install --upgrade foolbox



In [15]:
import foolbox as fb

# Define the DeepFool attack
attack = fb.attacks.L2DeepFoolAttack()  # Use 'LinfDeepFoolAttack()' for L∞ norm

def generate_deepfool_attack(face_tensor):
    raw_adversarial, clipped_adversarial, _ = attack(fmodel, face_tensor, criterion=fb.criteria.Misclassification())
    return clipped_adversarial


In [28]:
from PIL import Image
import torch
import torchvision.transforms as transforms
import numpy as np
import foolbox as fb
import torchvision.transforms as transforms
from PIL import Image
from facenet_pytorch import InceptionResnetV1

# Load an example image
image_path = "/content/premium_photo-1664203068007-52240d0ca48f.jpg"  # Replace with your image path
image = Image.open(image_path)

# Preprocess the image to match FaceNet's input size
transform = transforms.Compose([
    transforms.Resize((160, 160)),  # FaceNet expects 160x160 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

face_tensor = transform(image).unsqueeze(0)  # Add batch dimension
face_tensor = face_tensor.to(torch.float32)
facenet = InceptionResnetV1(pretrained='vggface2').eval()


In [29]:
def get_face_embedding(face_tensor):
    with torch.no_grad():
        return facenet(face_tensor).numpy().flatten()  # Convert to NumPy array


In [66]:

# Define the adversarial attack model
preprocessing = dict(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], axis=-3)
fmodel = fb.PyTorchModel(facenet, bounds=(-1, 1), preprocessing=preprocessing)

# Dummy label (Assuming classification task, set an actual label if available)
label = torch.tensor([0], dtype=torch.long)  # Modify based on real label

# FGSM Attack with epsilon value
def generate_fgsm_attack(face_tensor, label, epsilon=0.04):
    attack = fb.attacks.FGSM()
    raw_adversarial, clipped_adversarial, success = attack(fmodel, face_tensor, label, epsilons=epsilon)
    return clipped_adversarial
# DeepFool Attack
def generate_deepfool_attack(face_tensor, label, epsilon=100.0):
    attack = fb.attacks.L2DeepFoolAttack()
    raw_adversarial, clipped_adversarial, success = attack(fmodel, face_tensor, label, epsilons=epsilon)
    print("DeepFool Success:", success)  # Print success status
    print("DeepFool Difference:", (clipped_adversarial - face_tensor).abs().sum().item())
    return clipped_adversarial

def generate_pgd_attack(face_tensor, label):
    attack = fb.attacks.PGD()
    epsilons = [0.1]  # Adjust if needed
    raw_adversarial, clipped_adversarial, success = attack(
        fmodel, face_tensor, labels=label, epsilons=epsilons
    )

    print("PGD Success:", success)
    print("PGD Modification:", (clipped_adversarial - face_tensor).abs().sum().item())

    return clipped_adversarial
# Generate adversarial examples
fgsm_tensor = generate_fgsm_attack(face_tensor, label)
deepfool_tensor = generate_deepfool_attack(face_tensor, label)


DeepFool Success: tensor([True])
DeepFool Difference: 0.0


In [68]:
# Extract embeddings
original_embedding = get_face_embedding(face_tensor)
fgsm_embedding = get_face_embedding(fgsm_tensor)
deepfool_embedding = get_face_embedding(deepfool_tensor)

# Compute cosine similarity
def cosine_similarity(emb1, emb2):
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
# Print similarity scores
print("Original vs FGSM Attack Similarity:", cosine_similarity(original_embedding, fgsm_embedding))
print("Original vs DeepFool Attack Similarity:", cosine_similarity(original_embedding, deepfool_embedding))


Original vs FGSM Attack Similarity: 0.8960637
Original vs DeepFool Attack Similarity: 1.0


In [54]:
deepfool_tensor = generate_deepfool_attack(face_tensor, label)
print("DeepFool Modification:", (deepfool_tensor - face_tensor).abs().sum().item())



DeepFool Modification: 0.0
